In [1]:
import dask.dataframe as dd
names = ['time','source user@domain','destination user@domain','source computer','destination computer','authentication type','logon type','authentication orientation','success/failure','day']
df =  dd.read_csv('../lm-vol/LANL_test.csv', dtype='str', header= None, names=names, blocksize="1024MB")

In [6]:
df.head()

,time,source user@domain,destination user@domain,source computer,destination computer,authentication type,logon type,authentication orientation,success/failure,day
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success,1
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,1
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success,1
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success,1
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success,1


In [8]:
unique_df = dd.read_csv('../lm-vol/LANL_test_unique_v2.csv', dtype='str', blocksize="1024MB")

In [9]:
unique_df.head()

,source user@domain,destination computer,source computer
0,ANONYMOUS LOGON@C586,C586,C1250
1,ANONYMOUS LOGON@C586,C586,C586
2,C101$@DOM1,C988,C988
3,C1020$@DOM1,C1020,C1020
4,C1021$@DOM1,C625,C1021


In [12]:
duplicates = []
for index, row in unique_df.iterrows():
    dups = df[(df['source user@domain'] == row['source user@domain']) & (df['source computer'] == row['source computer']) & (df['destination computer'] == row['destination computer'])].shape[0].compute()
    duplicates.append(dups)
    print(index, dups)

0 3220
1 1351289
2 84449
3 1313
4 529
5 6747
6 6706
7 1342
8 25711
9 28290
10 1204
11 1159
12 662
13 614
14 1104
15 3102
16 1210


KeyboardInterrupt: 

In [89]:
import pandas as pd
names = ['time','source user@domain','destination user@domain','source computer','destination computer','authentication type','logon type','authentication orientation','success/failure','day']
max = 0
inter = 0
keys = []
unique_df = pd.read_csv('../lm-vol/LANL_test_unique_v2.csv', dtype='str')
for index, row in unique_df.iterrows():
    keys.append('['+row['source user@domain']+', '+row['source computer']+', '+row['destination computer']+']')
print(len(keys))

1062874


In [117]:
counts = dict.fromkeys(keys,0)

In [118]:
for chunk in pd.read_csv('../lm-vol/LANL_test.csv', dtype='str', header= None, names=names, chunksize=1000000):
    inter += 1
    df = chunk[['source user@domain', 'source computer', 'destination computer']]
    s = df.value_counts()
    for index, item in s.items():
        counts['['+index[0]+', '+index[1]+', '+index[2]+']'] += item

AttributeError: 'dict' object has no attribute 'astype'

In [127]:
temp = list(counts.keys())
res = list(map(lambda i: [i, counts[i]], temp))

In [133]:
import numpy as np
total_records = 0
for pair in res:
    total_records += pair[1]
print(total_records)
print(len(res))
res = np.array(res)

319239960
1062874


In [ ]:
import dask.dataframe as dd
names = ['time','source user@domain','destination user@domain','source computer','destination computer','authentication type','logon type','authentication orientation','success/failure','day']
test_df =  dd.read_csv('../lm-vol/LANL_test.csv', dtype='str', header= None, names=names, blocksize="1024MB")

In [134]:
np.save('../lm-vol/LANL_test_unique_freq.npy', res)

In [ ]:
persisteddf = df.persist()

In [ ]:
persisteddf.shape[0].compute()

In [ ]:
# df = df.assign(src_user = df['source user@domain'].str.split('@').map(lambda x: x[0]))
# df = df.assign(dest_user = df['destination user@domain'].str.split('@').map(lambda x: x[0]))

In [ ]:
# df = df[(df['src_user'].str.contains('LOCAL')==False) & (df['src_user'].str.contains('SYSTEM')==False) & (df['src_user'].str.contains('ANONYMOUS')==False) & (df['src_user'].str.contains('ADMINISTRARTOR')==False)]
# df = df[df['src_user'].str[-1] != '$']
# df = df[(df['dest_user'].str.contains('LOCAL')==False) & (df['dest_user'].str.contains('SYSTEM')==False) & (df['dest_user'].str.contains('ANONYMOUS')==False) & (df['dest_user'].str.contains('ADMINISTRARTOR')==False)]
# df = df[df['dest_user'].str[-1] != '$']

In [ ]:
df.head()

In [ ]:
df = df.drop_duplicates(subset=['source user@domain', 'destination computer', 'source computer'])

In [ ]:
LANL_train_unique_df = df[['source user@domain', 'destination computer', 'source computer']]

In [ ]:
LANL_train_unique_df.to_csv("../lm-vol/LANL_test_unique_v2.csv", index=False, single_file=True)

In [ ]:
LANL_train_unique_df.shape[0].compute()